This is the last step of evaluation. Take generated reponse json and feed this but make sure to hcange the tags in this code based on the response file

### Importing dependencies

In [47]:
import seqeval, json
import re 
import pandas

In [48]:
with open(r"G:\GitClone\BioNER_Project\llama2-ner\Evaluation\Generated_response\responses_bloom3b_gene_fast.json", "r") as f:
    raw_list=json.loads(f.read())
raw_list[0]

{'id': 0,
 'input': 'Do NER on this sentence\nThis stimulatory effect could also be elicited by c - Jun , which interacts with topo II , but not by c - Fos , which does not bind topo II in our in vitro assay\nReply in key value pair as Tag and Entity.',
 'expected': "['E-GENE:Jun', 'B-GENE:topo', 'E-GENE:II', 'E-GENE:Fos', 'B-GENE:topo', 'E-GENE:II']",
 'response': "['B-GENE:topo', 'I-GENE:II', 'E-GENE:alpha', 'B-GENE:c-Jun', 'E-GENE:Jun']"}

### Processing 

In [49]:
def tokenize(sentence):
    # Tokenization pattern similar to LLMs
    pattern = r"[\w']+|[.,!?;]"
    tokens = re.findall(pattern, sentence)
    return tokens

def extract_tags(tag_string):
    tags = []
    # Remove '[' and ']' and split by ','
    tag_string = tag_string.strip("[]")
    tag_string = tag_string.replace("'", "")
    tag_list = tag_string.split(", ")
    for tag in tag_list:
        # Split each tag into its key and value parts
        key, value = tag.split(":")
        tags.append((key.strip(), value.strip()))
    return tags

def get_true_and_pred(data):
    expected_tags = extract_tags(data['expected'])
    response_tags = extract_tags(data['response'])

    # Convert expected and response tags to y_true and y_pred respectively
    y_true = ['O'] * len(tokenize(data['input']))
    y_pred = ['O'] * len(tokenize(data['input']))

    # Check if expected_tags is empty
    if not expected_tags:
        for tag_type, tag_value in response_tags:
            # Find the indices where the tag value occurs in the input
            indices = [i for i, token in enumerate(tokenize(data['input'])) if token == tag_value]
            for index in indices:
                if tag_type == 'B-GENE'.lower():
                    y_pred[index] = 'B-B_GENE'
                elif tag_type == 'I-GENE'.lower():
                    y_pred[index] = 'I-I_GENE'
                elif tag_type == 'E-GENE'.lower():
                    y_pred[index] = 'E-E_GENE'
                elif tag_type == 'S-GENE'.lower():
                    y_pred[index] = 'S-S_GENE'
        return y_true, y_pred
    
    # Check if response_tags is empty
    if not response_tags:
        for tag_type, tag_value in expected_tags:
            # Find the indices where the tag value occurs in the input
            indices = [i for i, token in enumerate(tokenize(data['input'])) if token == tag_value]
            for index in indices:
                if tag_type == 'B-GENE'.lower():
                    y_true[index] = 'B-B_GENE'
                elif tag_type == 'I-GENE'.lower():
                    y_true[index] = 'I-I_GENE'
                elif tag_type == 'E-GENE'.lower():
                    y_true[index] = 'E-E_GENE'
                elif tag_type == 'S-GENE'.lower():
                    y_true[index] = 'S-S_GENE'
        return y_true, y_pred

    # Process expected_tags
    for tag_type, tag_value in expected_tags:
        # Find the indices where the tag value occurs in the input
        indices = [i for i, token in enumerate(tokenize(data['input'])) if token == tag_value]
        for index in indices:
            if tag_type == 'B-GENE'.lower():
                y_true[index] = 'B-B_GENE'
            elif tag_type == 'I-GENE'.lower():
                y_true[index] = 'I-I_GENE'
            elif tag_type == 'E-GENE'.lower():
                y_true[index] = 'E-E_GENE'
            elif tag_type == 'S-GENE'.lower():
                y_true[index] = 'S-S_GENE'

    # Process response_tags
    for tag_type, tag_value in response_tags:
        # Find the indices where the tag value occurs in the input
        indices = [i for i, token in enumerate(tokenize(data['input'])) if token == tag_value]
        for index in indices:
            if tag_type == 'B-GENE'.lower():
                y_pred[index] = 'B-B_GENE'
            elif tag_type == 'I-GENE'.lower():
                y_pred[index] = 'I-I_GENE'
            elif tag_type == 'E-GENE'.lower():
                y_pred[index] = 'E-E_GENE'
            elif tag_type == 'S-GENE'.lower():
                y_pred[index] = 'S-S_GENE'

    return y_true, y_pred

data = {
    'id': 0,
    'input': "Habitual use of acetaminophen as a risk factor for chronic renal failure : a comparison with phenacetin".lower(),
    'expected': "['B-GENE:chronic', 'I-GENE:renal', 'I-GENE:failure']".lower(),
    'response': "['B-GENE:]".lower()  # Empty response
}

y_true, y_pred = get_true_and_pred(data) # sample

print("y_true:", y_true)
print("y_pred:", y_pred)


y_true: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-B_GENE', 'I-I_GENE', 'I-I_GENE', 'O', 'O', 'O', 'O']
y_pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [50]:
y_true=[]
y_pred=[]
missed=0
for data in raw_list:
    obj={
        "id":data['id'],
        "input":data['input'].strip("Do NER on this sentence.").strip("Reply in key value pair as Tag and Entity.").lower().strip(),
        "expected":data['expected'].lower(),
        "response":data['response'].lower()
    }
    if data["expected"].strip()=="":
        obj["expected"]="['B-GENE:<>]"
    if data["response"].strip()=="":
        obj["response"]="['I-GENE:<>]"
        
    try:
        t,p=get_true_and_pred(obj)
        y_true.append(t)
        y_pred.append(p)
    except:
        missed+=1
        pass
print(f"Captured {len(raw_list)-missed}, and missed {missed}")

Captured 1474, and missed 26


- Missed a few responses because they did not clear validation of responses.

### Final part: 
- Printing the report

In [51]:
from seqeval.metrics import accuracy_score, precision_score, recall_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score

print("BLoom3b on BC2GM GENE Dataset 5epoch:\n")
print(f"Overall Accuracy Score : {round(accuracy_score(y_true, y_pred),2)} %")
print(f"       Precision Score : {round(precision_score(y_true, y_pred),2)} %")
print(f"          Recall Score : {round(recall_score(y_true, y_pred),2)} %")
print(f"              F1 Score : {round(f1_score(y_true, y_pred),2)} %",'\n')
print(classification_report(y_true, y_pred))

print(f"{'-'*100}\nSample object:\n {json.dumps(obj,indent=2)}")

BLoom3b on BC2GM GENE Dataset 5epoch:

Overall Accuracy Score : 0.84 %
       Precision Score : 0.52 %
          Recall Score : 0.24 %
              F1 Score : 0.33 % 

              precision    recall  f1-score   support

      B_GENE       0.60      0.40      0.48      2280
      E_GENE       0.57      0.27      0.37      2037
      I_GENE       0.21      0.10      0.13      1351
      S_GENE       0.61      0.07      0.13      1215

   micro avg       0.52      0.24      0.33      6883
   macro avg       0.50      0.21      0.28      6883
weighted avg       0.52      0.24      0.32      6883

----------------------------------------------------------------------------------------------------
Sample object:
 {
  "id": 1499,
  "input": "deletion analyses of the construct revealed that the transcription of boxf1 gene is regulated by box dna , preferentially in undifferentiated ec cells versus differentiated cells",
  "expected": "['b-gene:boxf1', 'e-gene:gene']",
  "response": "['b-ge

In [52]:
# y_true: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-B_GENE', 'I-I_GENE', 'I-I_GENE', 'O', 'O', 'O', 'O']
# y_pred: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# Captured 1474, and missed 26
# BLoom3b on BioNLP11D Species Dataset 5epoch:

# Overall Accuracy Score : 0.84 %
#        Precision Score : 0.52 %
#           Recall Score : 0.24 %
#               F1 Score : 0.33 % 

#               precision    recall  f1-score   support

#       B_GENE       0.60      0.38      0.46      2272
#       E_GENE       0.58      0.27      0.37      2037
#       I_GENE       0.22      0.10      0.14      1344
#       S_GENE       0.63      0.07      0.12      1214

#    micro avg       0.52      0.24      0.33      6867
#    macro avg       0.51      0.20      0.27      6867
# weighted avg       0.52      0.24      0.31      6867